# ⚡ Kvantisering (INT8) - Komprimera modellen för snabbare inference

**Mål**: Förstå hur kvantisering fungerar och när det är värt det.

I detta notebook kommer vi att:
- Förstå vad kvantisering är (FP32 → INT8)
- Se hur det påverkar modellstorlek och latens
- Experimentera med olika kalibreringsstorlekar
- Förstå kompromisser (accuracy vs prestanda)

> **💡 Tips**: Kvantisering är en av de viktigaste teknikerna för edge deployment - det kan göra modellen 4x snabbare!


## 🤔 Vad är kvantisering?

**Kvantisering** = konvertera modellen från 32-bit flyttal (FP32) till 8-bit heltal (INT8).

**Fördelar**:
- **4x mindre modellstorlek** (32 bit → 8 bit)
- **2-4x snabbare inference** (INT8 är snabbare att beräkna)
- **Mindre minnesanvändning** (viktigt för edge)

**Kompromisser**:
- **Accuracy-förlust** - modellen kan bli mindre exakt
- **Kalibrering krävs** - behöver representativ data för att hitta rätt skala

<details>
<summary>🔍 Klicka för att se tekniska detaljer</summary>

**Teknisk förklaring**:
- FP32: 32 bit per vikt (4 bytes)
- INT8: 8 bit per vikt (1 byte)
- Kvantisering hittar rätt skala för varje vikt
- Kalibrering använder representativ data för att optimera skalan

</details>


In [ ]:
# Först skapar vi en modell att kvantisera
print("🚀 Skapar modell för kvantisering...")
!python -m piedge_edukit.train --fakedata --no-pretrained --epochs 1 --batch-size 256 --output-dir ./models_quant


In [ ]:
# Kontrollera ursprunglig modellstorlek
import os

if os.path.exists("./models_quant/model.onnx"):
    original_size = os.path.getsize("./models_quant/model.onnx") / (1024*1024)
    print(f"📦 Ursprunglig modellstorlek: {original_size:.2f} MB")
else:
    print("❌ Modell saknas")


## 🧪 Experimentera med olika kalibreringsstorlekar

**Kalibreringsstorlek** = antal bilder som används för att hitta rätt skala för kvantisering.

**Större kalibrering**:
- ✅ Bättre accuracy (mer representativ data)
- ❌ Längre kvantiserings-tid
- ❌ Mer minne under kvantisering

**Mindre kalibrering**:
- ✅ Snabbare kvantisering
- ✅ Mindre minnesanvändning
- ❌ Potentiellt sämre accuracy


In [ ]:
# Test 1: Liten kalibrering (snabb)
print("⚡ Test 1: Liten kalibrering (16 bilder)")
!python -m piedge_edukit.quantization --fakedata --model-path ./models_quant/model.onnx --calib-size 16


In [ ]:
# Visa kvantiseringsresultat
if os.path.exists("./reports/quantization_summary.txt"):
    with open("./reports/quantization_summary.txt", "r") as f:
        print("📊 Kvantiseringsresultat:")
        print(f.read())
else:
    print("❌ Kvantiseringsrapport saknas")


In [ ]:
# Jämför modellstorlekar
if os.path.exists("./models_quant/model.onnx") and os.path.exists("./models_quant/model_static.onnx"):
    original_size = os.path.getsize("./models_quant/model.onnx") / (1024*1024)
    quantized_size = os.path.getsize("./models_quant/model_static.onnx") / (1024*1024)
    
    print(f"📦 Modellstorlekar:")
    print(f"  Ursprunglig (FP32): {original_size:.2f} MB")
    print(f"  Kvantiserad (INT8): {quantized_size:.2f} MB")
    print(f"  Komprimering: {original_size/quantized_size:.1f}x")
else:
    print("❌ Modellfiler saknas")


In [ ]:
# Benchmark båda modellerna för att jämföra latens
print("🚀 Benchmark ursprunglig modell (FP32)...")
!python -m piedge_edukit.benchmark --fakedata --model-path ./models_quant/model.onnx --warmup 3 --runs 20 --providers CPUExecutionProvider


In [ ]:
# Benchmark kvantiserad modell (INT8)
print("⚡ Benchmark kvantiserad modell (INT8)...")
!python -m piedge_edukit.benchmark --fakedata --model-path ./models_quant/model_static.onnx --warmup 3 --runs 20 --providers CPUExecutionProvider


In [ ]:
# Jämför latens-resultat
import pandas as pd

# Läs båda benchmark-resultaten
fp32_file = "./reports/latency_summary.txt"
if os.path.exists(fp32_file):
    with open(fp32_file, "r") as f:
        fp32_content = f.read()
    
    # Extrahera mean latens från texten (enkel parsing)
    lines = fp32_content.split('\n')
    fp32_mean = None
    for line in lines:
        if 'Mean' in line and 'ms' in line:
            try:
                fp32_mean = float(line.split(':')[1].strip().replace('ms', '').strip())
                break
            except:
                pass
    
    print(f"📊 Latens-jämförelse:")
    if fp32_mean:
        print(f"  FP32 (ursprunglig): {fp32_mean:.2f} ms")
    else:
        print(f"  FP32: Kunde inte parsa latens")
    
    # TODO: Lägg till INT8-latens här när den är tillgänglig
    print(f"  INT8 (kvantiserad): [kommer efter benchmark]")
else:
    print("❌ Benchmark-rapport saknas")


## 🤔 Reflektionsfrågor

<details>
<summary>💭 När är INT8-kvantisering värt det?</summary>

**Svar**: INT8 är värt det när:
- **Latens är kritisk** - realtidsapplikationer, edge deployment
- **Minne är begränsat** - mobil, Raspberry Pi
- **Accuracy-förlusten är acceptabel** - < 1-2% accuracy-förlust är ofta OK
- **Batch size är liten** - kvantisering fungerar bäst med små batches

**När INTE värt det**:
- Accuracy är absolut kritisk
- Du har gott om minne och CPU
- Modellen är redan snabb nog

</details>

<details>
<summary>💭 Vilka risker finns med kvantisering?</summary>

**Svar**: Huvudrisker:
- **Accuracy-förlust** - modellen kan bli mindre exakt
- **Kalibreringsdata** - behöver representativ data för bra kvantisering
- **Edge cases** - extrema värden kan orsaka problem
- **Debugging** - kvantiserade modeller är svårare att debugga

**Minskning**:
- Testa noggrant med riktig data
- Använd olika kalibreringsstorlekar
- Benchmark både accuracy och latens

</details>


## 🎯 Ditt eget experiment

**Uppgift**: Testa olika kalibreringsstorlekar och jämför resultaten.

**Förslag**:
- Testa kalibreringsstorlekar: 8, 16, 32, 64
- Jämför modellstorlek och latens
- Analysera accuracy-förlust (om tillgänglig)

**Kod att modifiera**:
```python
# Ändra dessa värden:
CALIB_SIZE = 32

!python -m piedge_edukit.quantization --fakedata --model-path ./models_quant/model.onnx --calib-size {CALIB_SIZE}
```


In [ ]:
# TODO: Implementera ditt experiment här
# Ändra värdena nedan och kör kvantisering

CALIB_SIZE = 32

print(f"🧪 Mitt experiment: kalibreringsstorlek={CALIB_SIZE}")

# TODO: Kör kvantisering med din inställning
# !python -m piedge_edukit.quantization --fakedata --model-path ./models_quant/model.onnx --calib-size {CALIB_SIZE}


## 🎉 Sammanfattning

Du har nu lärt dig:
- Vad kvantisering är (FP32 → INT8) och varför det är viktigt
- Hur kalibreringsstorlek påverkar resultatet
- Kompromisser mellan accuracy och prestanda
- När kvantisering är värt det vs när det inte är det

**Nästa steg**: Gå till `04_evaluate_and_verify.ipynb` för att förstå automatiska checks och kvittogenerering.

**Viktiga begrepp**:
- **Kvantisering**: FP32 → INT8 för snabbare inference
- **Kalibrering**: Representativ data för att hitta rätt skala
- **Komprimering**: 4x mindre modellstorlek
- **Speedup**: 2-4x snabbare inference
